In [2]:
import tensorflow as tf
from tensorflow.keras import layers

c:\users\usherwoodpe\appdata\local\continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
tf.__version__

'2.0.0'

In [4]:
x = tf.random.uniform([3, 3])

In [5]:
print("Is there a GPU available: "),
print(tf.test.is_gpu_available())

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

print("Device name: {}".format((x.device)))

Is there a GPU available: 
True
Is the Tensor on GPU #0:  
True
Device name: /job:localhost/replica:0/task:0/device:GPU:0


In [6]:
import os
import sys
import sys
import os

# Add the path to system, local or mounted S3 bucket, e.g. /dbfs/mnt/<path_to_bucket>
sys.path.append(os.path.join(os.getcwd(),"bert"))
import numpy as np
import json
import nltk
import pandas as pd
import csv
import random
import logging
from collections import Counter
import pathlib
import pickle

import modeling, optimization, tokenization
from run_pretraining import input_fn_builder, model_fn_builder

from text_preprocessing import tokenizer_word
from language_model_processing import read_raw_data_preprocess_and_save, create_vocab_df
from bpe import create_token_vocabulary, get_stats, merge_vocab, Encoder

In [7]:
language_maps_dir = "models/base/master/language_maps/"

def save_obj(obj, directory, name):
    with open(directory / "{}.pkl".format(name), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name, directory):
    with open(os.path.join(directory, name + '.pkl'), 'rb') as f:
        return pickle.load(f)
      
vocab_to_id = load_obj('vocab_to_id', str(language_maps_dir))
print('Vocab Size:', len(vocab_to_id))

import json

bert_base_config = {
  "attention_probs_dropout_prob": 0.1, 
  "directionality": "bidi", 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 768, 
  "initializer_range": 0.02, 
  "intermediate_size": 3072, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 12, 
  "num_hidden_layers": 12, 
  "pooler_fc_size": 768, 
  "pooler_num_attention_heads": 12, 
  "pooler_num_fc_layers": 3, 
  "pooler_size_per_head": 128, 
  "pooler_type": "first_token_transform", 
  "type_vocab_size": 2, 
  "vocab_size": len(vocab_to_id)
}

with open(os.path.join(language_maps_dir, 'bert_config.json'), 'w') as f:
    json.dump(bert_base_config, f)
    
print(bert_base_config)
####################################load_vocab

Vocab Size: 31503
{'attention_probs_dropout_prob': 0.1, 'directionality': 'bidi', 'hidden_act': 'gelu', 'hidden_dropout_prob': 0.1, 'hidden_size': 768, 'initializer_range': 0.02, 'intermediate_size': 3072, 'max_position_embeddings': 512, 'num_attention_heads': 12, 'num_hidden_layers': 12, 'pooler_fc_size': 768, 'pooler_num_attention_heads': 12, 'pooler_num_fc_layers': 3, 'pooler_size_per_head': 128, 'pooler_type': 'first_token_transform', 'type_vocab_size': 2, 'vocab_size': 31503}


In [8]:
import modeling, optimization, tokenization

testcase = "Olá isso é mais uma BAGUNCA 😂😂😂"
bert_tokenizer = tokenization.FullTokenizer(language_maps_dir)
print(testcase)
print(bert_tokenizer.tokenize(testcase))

Olá isso é mais uma BAGUNCA 😂😂😂
['olá', 'isso', 'é', 'mais', 'uma', 'bagun', 'ca', '😂', '😂', '😂']


In [143]:
model_weights_dir = "models/base/master/model_weights/"
pretraining_data_dir = 'models/base/master/pretraining_base_data'

VOCAB_FILE = language_maps_dir + '/vocab_file.csv'
CONFIG_FILE = language_maps_dir + '/bert_config.json'

INIT_CHECKPOINT = tf.train.latest_checkpoint(model_weights_dir)

MINI_BATCH_SIZE = 32

bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)
input_files = tf.io.gfile.glob(os.path.join(pretraining_data_dir,'*tfrecord'))

In [151]:
def tf_record_to_memory(input_files,
                        max_seq_length = 128,
                        max_predictions_per_seq = 20,
                        num_cpu_threads = 4,
                        batch_size = MINI_BATCH_SIZE):
    
    d = tf.data.TFRecordDataset(input_files)

    name_to_features = {
        "input_ids":
            tf.io.FixedLenFeature([max_seq_length], tf.int64),
        "input_mask":
            tf.io.FixedLenFeature([max_seq_length], tf.int64),
        "segment_ids":
            tf.io.FixedLenFeature([max_seq_length], tf.int64),
        "masked_lm_positions":
            tf.io.FixedLenFeature([max_predictions_per_seq], tf.int64),
        "masked_lm_ids":
            tf.io.FixedLenFeature([max_predictions_per_seq], tf.int64),
        "masked_lm_weights":
            tf.io.FixedLenFeature([max_predictions_per_seq], tf.float32),
        "next_sentence_labels":
            tf.io.FixedLenFeature([1], tf.int64),
    }
    parsed_dataset = d.map(
        map_func = lambda record: tf.io.parse_single_example(record, name_to_features),
        num_parallel_calls=num_cpu_threads)
    parsed_dataset = parsed_dataset.batch(drop_remainder=True, 
                                          batch_size=batch_size)
    
    return parsed_dataset

def get_single_sentence_training_data(i, s):
    input_ids = tf.boolean_mask(
        i,
        1-s,
        axis=None,
        name='boolean_mask')
    x = tf.slice(input_ids, [1], [input_ids.shape[0]-2], name='x')
    x = tf.pad(x, [[0, 128-input_ids.shape[0]+2]], "CONSTANT", constant_values=0)
    y = input_ids[-2]
        
    return x, y

In [132]:
model = tf.keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
model.add(layers.Embedding(input_dim=len(vocab_to_id), output_dim=64))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units and softmax activation.
model.add(layers.Dense(len(vocab_to_id), activation='softmax'))

model.summary()
    
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), #tf.keras.loss
              optimizer='adam', #tf.keras.optimizers
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          2016192   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 31503)             4063887   
Total params: 6,178,895
Trainable params: 6,178,895
Non-trainable params: 0
_________________________________________________________________


In [153]:
model(X)

UnknownError: Fail to find the dnn implementation. [Op:CudnnRNN]

In [136]:
for bn, batch in enumerate(parsed_dataset.repeat().batch(MINI_BATCH_SIZE).take(MINI_BATCH_SIZE)):
    X = []
    Y = []
    ips = batch['input_ids']
    sms = batch['segment_ids']
    for i, s in zip(ips, sms):
        x, y = get_single_sentence_training_data(i, s)
        X.append(x)
        Y.append(y)
    X = tf.stack(X)
    Y = tf.stack(Y)
    

tf.Tensor(
[ 1500  1624  3089  1581  1542  1604  2452  1530  3530    12    69  1604
  1502  1596    14    63 27807    63  5089  1502  1603  6665 12543  1800
 11915  1540 15672  1506  8796  1511  3132  1506  1502  1583  2877  7651
 15086    14  1604  2494  2988  6932  2191  1502  1674  2324  2639    12
 27056    69  6522  1563  1973    14  1501   168  2188  1624  1502  1720
    13    63 27807    63  1547    63  1720 24468  1502    63  1547    63
  3260    63 27807    63  1502    63  3042  1524  1502 27807    63  1547
    63  1571  1935    63 27807  1502  1547    63  2702    13  1651  1624
  1551    70    13  1639    63 27807  1502  1547    63  3260    63 27807
    63  1547    63  5170  1530  1502 27807    63  1547  1502  5651  2965
  1502 27807    63  5089    63  1502  1604  1501], shape=(128,), dtype=int64)
tf.Tensor(
[ 1500    63 27807    63  5089  1502  1603  1502  3185  3213  3378  6323
  1502  3378  1846    14  2963  7223    14  2571 15475    14  1651  2452
    14  2452  7047  2625